In [1]:
from IPython.display import HTML
#DOESN'T SEEM T WORK
HTML("""
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
""")
#aim is to autorun all

In [2]:
from Jumpscale import j
simulation = j.tools.tfsimulation.simulation_get()
environment = simulation.environment_get("main")
from IPython.display import Markdown as md
import numpy as np
import bqplot.pyplot as plt
from bqplot import *
from ipywidgets import interact
import ipywidgets as widgets
from ipywidgets import HBox, Label, Layout


## input arguments

In [3]:
environment.cost_power_kwh = "0.15 USD"
environment.cost_rack_unit = "12 USD"
environment.bandwidth_mbit = 20

# sales arguments
environment.sales_price_cu = "8 USD"
environment.sales_price_su = "5 USD"

device_edge = simulation.device_get("edge1", environment=environment)
# print(d1)

switch = simulation.device_get("switch", environment=environment)

environment.device_add("edge1", device_edge, 20)
environment.device_add("switch", switch, 2)

# print(environment)

simulation_run = simulation.run()

# month:growth_percent of nodes being added
simulation_run.growth_percent_set("3:5,11:8,24:10,36:12,48:5,60:5")

# means at end of period we produce 40% more cpr (*1.4)
# cpr = capacity production rate (is like hashrate of bitcoin)
simulation_run.cpr_improve_set("71:40")

# price of a capacity unit goes down over time, here we say it will go down 40%
# means we expect price to be lowered by X e.g. 40 (*0.6)
simulation_run.cpr_sales_price_decline_set("0:100,71:40")

# utilization of the nodes, strating with 0
simulation_run.utilization_set("30:80,71:90")

# super important factor, how does token price goes up, this is ofcourse complete speculation, no-one knows
simulation_run.tokenprice_set("0:0.15,30:2,71:2.5")

# set difficulty level
simulation_run.difficulty_level_set("0:2,71:50")

# are the parameters for a batch of nodes to be added
nb = simulation_run.nodes_batch_template_get(environment)
# print(nb)

# do the calculation of the simulation
simulation_run.calc(nr_start_nodes=1500, months_remaining_start_nodes=36,environment=environment)


error: 'I' format requires 0 <= number <= 4294967295

In [ ]:
C="""
```python
%s
```
"""%In[2]
m=md(C)
m

### calculated edge device

In [ ]:
device_edge

### environment of 1500 edge devices

Is a complete environment with switches and 1500 edge devices


In [ ]:
environment

### Growth Percent per month

Choose Number of months



In [ ]:
def plotGrowth(nrmonths):
    plt.figure(title='Growth Percent Per Month (%s Months)' % nrmonths)
    plt.plot(np.arange(nrmonths), simulation_run.sheet.rows["growth_percent"].cells[:nrmonths])
    plt.show()
    
interact(plotGrowth, nrmonths=widgets.IntSlider(min=24, max=72, step=24, value=72, description='Nr months'))


### Number of nodes per month

Choose Number of months



In [ ]:
def plotNodeGrowth(nrmonths):
    plt.figure(title='Nr Nodes (%s Months)' % nrmonths)
    plt.plot(np.arange(nrmonths), simulation_run.sheet.rows["nrnodes"].cells[:nrmonths])
    plt.show()

In [ ]:
interact(plotNodeGrowth, nrmonths=widgets.IntSlider(min=24, max=72, step=24, value=72, description='Nr months'))

### Number of racks per month

Choose Number of months


In [ ]:
def plotRackGrowth(nrmonths):
    plt.figure(title='Nr Racks Occupied. (%s Months)' % nrmonths)
    plt.plot(np.arange(nrmonths), simulation_run.sheet.rows["nrracks"].cells[:nrmonths])
    plt.show()


In [ ]:
interact(plotRackGrowth, nrmonths=widgets.IntSlider(min=24, max=72, step=24, value=72, description='Nr months'))

### Revenue per month

Choose Number of months


In [ ]:
def revenuGrowth(nrmonths):
    plt.figure(title='Revenue Per Month (kUSD). (%s Months)'% nrmonths)
    plt.plot(np.arange(nrmonths), [int(i/1000) for i in simulation_run.sheet.rows["revenue_month"].cells[:nrmonths]])
    plt.show()

In [ ]:
interact(revenuGrowth, nrmonths=widgets.IntSlider(min=24, max=72, step=24, value=72, description='Nr months'))

### Revenu per year

In [ ]:
plt.figure(title='Revenue Per Year (million USD).')
plt.bar(np.arange(0,6), [int(i/1000) for i in simulation_run.sheet.rows["revenue_month"].aggregate("Y")])
plt.show()

### Revenu over 60 months

In [ ]:
plt.figure(title='Recurring Revenue Over 60 Months (million USD).')
plt.bar(np.arange(simulation_run.sheet.nrcols), [round(float(i)*60/1000,2) for i in simulation_run.sheet.rows["revenue_month"].cells])
plt.show() 

### Token price per month
Choose Number of months

In [ ]:
def plotTokenPrice(nrmonths):
    plt.figure(title='Token Price (TFT). (%s months)' % nrmonths)
    plt.plot(np.arange(nrmonths), [round(i,2) for i in simulation_run.sheet.rows["tokenprice"].cells[:nrmonths]])
    plt.show()

In [ ]:
interact(plotTokenPrice, nrmonths=widgets.IntSlider(min=24, max=72, step=24, value=72, description='Nr months'))

In [ ]:
simulation_run